In [1]:
from importlib import reload
import flymovie as fm
import cnn_models.siamese_cnn as cn
import cnn_models.evaluate_models as ev
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
from importlib import reload
import scipy.ndimage as ndi
import skimage as ski
import pandas as pd
%load_ext autoreload
%autoreload 2

In [23]:
def load_folder(folder):
    ims = []
    for f in os.listdir(folder):
        if f[0] == '.':
            continue
        im = fm.load_pickle(os.path.join(folder, f))
        im = im_norm(im)
        ims.append(im)
    return np.array(ims)

def im_norm(x):
    im = x.copy()
    nz = im[im>0]
    min_ = np.min(nz)
    max_ = np.max(nz)
    im[im == 0] = min_
    
    return (im - min_) / (max_ - min_) * 1000

def make_mask_file(ims, outfilepath, struct=np.ones((1,7,7)), target_dims=(100,100,100)):
    masks = []
    for im in ims:
        mask = np.where(im > 0, 1, 0)
        mask = ndi.morphology.binary_erosion(mask, struct)
        mask = ndi.zoom(mask, 
            (target_dims[0] / mask.shape[0],
            target_dims[1] / mask.shape[1],
            target_dims[2] / mask.shape[2],
            ),
            order=0)
        masks.append(mask.astype(bool))
    masks = np.array(masks)
    fm.save_pickle(masks, outfilepath)

def knorm(k):
    return (k - np.min(k)) / (np.max(k) - np.min(k))

### ZLD WT

In [15]:
reals_folder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_nuclei/set2/tmp/zld-wt-early13'
masks_file = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_masks/mask_files/zld-wt_earlync13_masks.pkl'

reals = load_folder(reals_folder)
masks_src = fm.load_pickle(masks_file)

In [5]:
kernel = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220512_nm100x50x50_px27x25x25.pkl')
kernel = knorm(kernel)

outfolder = '/Users/michaelstadler/Desktop/temp'
nreps = 1
concentration = 40
hlb_diam_rng = [12,12]
hlb_nmols_rng = [1,1]
n_clusters_rng = [315, 315]
cluster_diam_mean_rng = [1,1] 
cluster_diam_var_rng = [0.001,0.001]
cluster_nmols_mean_rng = [6,6]
cluster_nmols_var_rng = [1.3,1.3]
noise_sigma_rng = [3.43, 3.43]
dims_init=(85, 85, 85)
dims_kernel=(100,50,50)
dims_final=(250,85,85)
gfp_intensity = 2.7

#hlb_coords = [hlb_possible_coords[x] for x in np.random.choice(len(hlb_possible_coords), 2)]
hlb_coords = [(12, 60, 38), (51, 57, 41)]
sims = []
nsims=1
for _ in range(nsims):
    masks = masks_src[np.random.choice(np.arange(masks_src.shape[0]), nsims)]
    sim = fm.sim_rpb1(masks, kernel, outfolder, nreps, concentration, hlb_diam_rng, 
        hlb_nmols_rng, n_clusters_rng, cluster_diam_mean_rng, 
        cluster_diam_var_rng, cluster_nmols_mean_rng, cluster_nmols_var_rng,
        noise_sigma_rng, hlb_coords, dims_init, 
        dims_kernel, dims_final, gfp_intensity, return_sim=True, mask_nuclei=True)
    im = im_norm(sim.im.copy())

for _ in range(reals.shape[0]):
    sims.append(im)
sims = np.array(sims)

fm.viewer([reals, sims], 5)

interactive(children=(Dropdown(description='Color', index=9, options=('inferno', 'Gators', 'viridis', 'Greens'…

### ZLD c-term

In [58]:
reals_folder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_nuclei/set2/tmp/zld-cterm-early/'
masks_file = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_masks/mask_files/zld_cterm_early.pkl'

reals = load_folder(reals_folder)
masks_src = fm.load_pickle(masks_file)

In [77]:
#kernel = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220506_100nmx50x50_53pxx21x21_lobestretch_17_3.0.pkl')
kernel = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220512_nm100x50x50_px27x25x25.pkl')
outfolder = '/Users/michaelstadler/Desktop/temp'
nreps = 1
concentration = 25
hlb_diam_rng = [12,12]
hlb_nmols_rng = [0,0]
n_clusters_rng = [0, 0]
cluster_diam_mean_rng = [1,1] 
cluster_diam_var_rng = [0.01,0.01]
cluster_nmols_mean_rng = [1,1]
cluster_nmols_var_rng = [0.001,0.001]
noise_sigma_rng = [6, 6]
dims_init=(85, 85, 85)
dims_kernel=(100,50,50)
dims_final=(250,85,85)
gfp_intensity = 2.7

#hlb_coords = [hlb_possible_coords[x] for x in np.random.choice(len(hlb_possible_coords), 2)]
hlb_coords = [(12, 60, 38), (51, 57, 41)]
sims = []
nsims=1
for _ in range(nsims):
    masks = masks_src[np.random.choice(np.arange(masks_src.shape[0]), nsims)]
    sim = fm.sim_rpb1(masks, kernel, outfolder, nreps, concentration, hlb_diam_rng, 
        hlb_nmols_rng, n_clusters_rng, cluster_diam_mean_rng, 
        cluster_diam_var_rng, cluster_nmols_mean_rng, cluster_nmols_var_rng,
        noise_sigma_rng, hlb_coords, dims_init, 
        dims_kernel, dims_final, gfp_intensity, return_sim=True, mask_nuclei=True)
    im = im_norm(sim.im.copy())

for _ in range(reals.shape[0]):
    sims.append(im)
sims = np.array(sims)

fm.viewer([reals, sims], 5)
#3.8-5.2

interactive(children=(Dropdown(description='Color', index=9, options=('inferno', 'Gators', 'viridis', 'Greens'…

In [53]:
infolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_nuclei/set2/tmp/zld-cterm-early'
masks = []
for f in os.listdir(infolder):
    if f[0] == '.':
        continue
    im = fm.load_pickle(os.path.join(infolder, f))
    mask = np.where(im > 0, 1, 0)
    mask = ndi.morphology.binary_erosion(mask, np.ones((1,7,7)))
    mask = ndi.zoom(mask, (100/34,1, 1), order=0)
    masks.append(mask.astype(bool))


In [106]:
def maskfile_from_masked_ims(im_folder_path, outpath, target_dims=(100,100,100), erosion_struct=(1,7,7)):
    """Make a maskfile from a folder of masked images.
    
    Assumes that all pixel values of 0 are background and all
    foreground pixels are > 0.

    Args:
        im_folder_path: str
            Path to folder containing images (pickled ndarrays)
        outpath: str
            Path to write pickle file to
        target_dims: tuple of 3 ints
            Dimensions of final masks
        erosion_structure: tuple of 3 ints
            Structure used for morphological dilation in constructing
            input images (this is reversed by erosion).
    """
    masks = []
    for f in os.listdir(im_folder_path):
        if f[0] == '.':
            continue
        im = fm.load_pickle(os.path.join(im_folder_path, f))
        mask = np.where(im > 0, 1, 0)
        if erosion_struct is not None:
            mask = ndi.morphology.binary_erosion(mask, np.ones(erosion_struct))
        mask = ndi.zoom(mask, np.divide(target_dims, im.shape), order=0)
        masks.append(mask.astype(bool))
    masks = np.array(masks)
    save_pickle(masks, outpath)

maskfile_from_masked_ims('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_nuclei/set2/tmp/zld-wt-early13', '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_masks/mask_files/zld-wt_earlync13_masks2.pkl')


In [107]:
m1 = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_masks/mask_files/zld-wt_earlync13_masks.pkl')
m2 = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_masks/mask_files/zld-wt_earlync13_masks2.pkl')
np.array_equal(m1, m2)

True